# Dynamics

This file contains the functions for moving the particles. 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import gamma
import time
import math
import matplotlib as mp
import scipy as sp
import pylab as py
import os
import matplotlib.animation as animation

from scipy.integrate import odeint

plt.rcParams['axes.grid'] = True
plt.style.use('dark_background')

In [2]:
G = 6.67259e-11  # (km**3/kg/s**2)
Ms = 1.98847e30 # solar mass in kg
c = 3e8

In [2]:
def grav_force(mass_1, mass_2, r):
    # provides the force as a (x,y,z) array 
    # must be given two masses and a (x,y,z) distance array
    return r*(G*mass_1*mass_2)/(np.linalg.norm(r)**3)

def get_r(pos1, pos2):
    # provides the (x,y,z) distance array
    # requires pos1 and pos2 to be (x,y,z) arrays
    r_vec = np.array([pos1[0]-pos2[0], pos1[1]-pos2[1], pos1[2]-pos2[2]])
    return r_vec

def gwpl(m1, m2, r):
    """Gravitational wave power loss. 
    r must be a (x,y,z) array 
    P is power """
    P = ((G**4)*(m1+m2)*(m1*m2)**2)/((c*np.linalg.norm(r))**5)
    return P

def leapfrogintegrate(pos1, pos2, vel1, vel2, mass1, mass2, N=35000, dt=2000):
    # pos1 = [x,y,z] array
    # pos2 = [x,y,z] array
    # vel1 = [x,y,z] array
    # vel2 = [x,y,z] array
    
    # does leapfrog integration and returns position and velocity values 
    
    # Create empty arrays 

    p1 = np.zeros([N,3])
    p2 = np.zeros([N,3])
    v1 = np.zeros([N,3])
    v2 = np.zeros([N,3])
    
    # initialize 
    for index in range(N-1):
        # set initial step 
        if index == 0:
            p1[0] = pos1
            p2[0] = pos2
            v1[0] = vel1
            v2[0] = vel2
        # integrate 
        else:
            r = get_r(p1[index-1], p2[index-1])
            F = grav_force(mass1, mass2, r)
            # sets next velocity. 
            # uses the particle's acceleration, last jump, and grav force
            v1[index] = v1[index-1]-dt*F/mass1
            v2[index] = v2[index-1]+dt*F/mass2
            
            # sets next position. 
            # uses the last jump and next velocity 
            p1[index] = p1[index-1]+dt*v1[index]
            p2[index] = p2[index-1]+dt*v2[index]
    
    return p1,p2,v1,v2

In [4]:
def orbit(ns1, ns2, d=1.5e11, tend=2200):
    
    p1 = np.zeros([tend,3])
    p1[0] = np.array([d,0,0])
    m1 = ns1.get_mass()*Ms
    p2 = np.zeros([tend,3])
    p2[0] = np.array([0,0,0])
    m2 = ns2.get_mass()*Ms

    # make elliptical orbit in xz plane
    ev = (2*3.14159*d/(365*24*60*60)) 
    mom1 = np.zeros([tend,3])
    mom1[0] = np.array([0, m1*ev, 0])
    mom2 = np.zeros([tend,3])
    mom2[0] = -1*mom1[0]
    dt = 12*60*60

    t = 0
    i = 1
    while t<tend-1:
        r = p2[i-1]-p1[i-1]
        F = -(G*m2*m1*r/np.linalg.norm(r)**3)
        mom2[i] = mom2[i-1] + F*dt
        p2[i] = p2[i-1] + (mom2[i]/m2)*dt
        mom1[i] = mom1[i-1] - F*dt
        p1[i] = p1[i-1] + (mom1[i]/m1)*dt
        t = t+1
        i = i+1
        
    return p1, p2

In [ ]:
def grav_drag(m1, m2, r):
    #rdot = (-64/5)*(m1*m2*(m1+m2))/(np.linalg.norm(r)**5)
    rdot = (-0.005)*(m1*m2*(m1+m2))*r/(np.linalg.norm(r)**6)
    return rdot

def leapfrogintegrate_drag(pos1, pos2, vel1, vel2, mass1, mass2, N=17800, dt=5000):
    # pos1 = [x,y,z] array
    # pos2 = [x,y,z] array
    # vel1 = [x,y,z] array
    # vel2 = [x,y,z] array
    
    # does leapfrog integration and returns position and velocity values 
    
    # Create empty arrays 

    p1 = np.zeros([N,3])
    p2 = np.zeros([N,3])
    v1 = np.zeros([N,3])
    v2 = np.zeros([N,3])
    
    # initialize 
    for index in range(N):
        # set initial step 
        if index == 0:
            p1[0] = pos1
            p2[0] = pos2
            v1[0] = vel1
            v2[0] = vel2
        # integrate 
        else:
            r = get_r(p1[index-1], p2[index-1])
            if np.linalg.norm(r) < 40928579265:
                p1[index] = p1[index-1]
                p2[index] = p1[index]
                
            else:
                F = grav_force(mass1, mass2, r)
                gwdrag = grav_drag(mass1,mass2, r)
                # sets next velocity. 
                # uses the particle's acceleration, last jump, and grav force
                v1[index] = v1[index-1]-dt*F/mass1 
                v2[index] = v2[index-1]+dt*F/mass2

                # sets next position. 
                # uses the last jump and next velocity 
                p1[index] = p1[index-1]+dt*(v1[index]+gwdrag/mass1)
                p2[index] = p2[index-1]+dt*(v2[index]+gwdrag/mass2)
    
    return p1,p2,v1,v2